In [ ]:
#Found that the original latitude and longitude were not stored to be referenced in merge.
import os
from dotenv import load_dotenv
import requests
import pandas as pd

# Load environment variables
load_dotenv('D:/Documents/GitHub/Statistical-Modelling-Project/notebooks/.env')

# Declare API key and client info
FOURSQUARE_API_KEY = os.getenv("FOURSQUARE_API_KEY")
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

# URL for places query
base_url = 'https://api.foursquare.com/v3/places/search'

# Load dataframe from CSV
montreal_bike_data = pd.read_csv('D:/Documents/GitHub/Statistical-Modelling-Project/notebooks/montreal_df.csv')


# Initialize empty list to store data for DataFrame
data_for_df = []

# Parameters for API request
params = {
    'query': 'restaurant',
    'radius': 1000,
    'limit': 10,
    'sort': 'rating'
}

headers = {
    "Accept": "application/json",
    "Authorization": FOURSQUARE_API_KEY
}

# Loop through dataframe to make API requests
for index, row in montreal_bike_data.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    ll_joined = f"{latitude},{longitude}"
    params['ll'] = ll_joined

    try:
        response = requests.get(base_url, params=params, headers=headers, timeout=30)
        response.raise_for_status()

        result_data = response.json()
        
        # Extract relevant information from the result_data
        for result in result_data.get('results', []):
            venue_name = result.get('name')
            address = result['location'].get('formatted_address')
            venue_latitude = result['geocodes']['main']['latitude']  # Latitude of the venue
            venue_longitude = result['geocodes']['main']['longitude']  # Longitude of the venue
            categories = [cat['name'] for cat in result['categories']]
            rating = result.get('rating', None)
            total_ratings = result['stats']['total_ratings'] if 'stats' in result else None
        
            # Append extracted data to data_for_df as a dictionary
            data_for_df.append({
                'Original_Latitude': latitude,  # Original latitude from montreal_bike_data
                'Original_Longitude': longitude,  # Original longitude from montreal_bike_data
                'Venue_Name': venue_name,
                'Address': address,
                'Latitude': venue_latitude,  # Latitude of the venue from Foursquare
                'Longitude': venue_longitude,  # Longitude of the venue from Foursquare
                'Categories': categories,
                'Rating': rating,
                'TotalRatings': total_ratings
            })
    
    except requests.exceptions.RequestException as e:
        print(f"Request failed for latitude {latitude}, longitude {longitude}. Error: {e}")

# Convert the list of dictionaries into a DataFrame
foursquare_df = pd.DataFrame(data_for_df)

# Display the DataFrame
print(foursquare_df.head())

# Save DataFrame to CSV
foursquare_df.to_csv('foursquare_data.csv', index=False)

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
#This is Snippet of full code above
# Parameters for API request
params = {
    'query': 'restaurant',
    'radius': 1000,
    'limit': 10,
    'sort': 'rating'
}

headers = {
    "Accept": "application/json",
    "Authorization": FOURSQUARE_API_KEY
}

# Loop through dataframe to make API requests
for index, row in montreal_bike_data.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    ll_joined = f"{latitude},{longitude}"
    params['ll'] = ll_joined

    try:
        response = requests.get(base_url, params=params, headers=headers, timeout=30)
        response.raise_for_status()

        result_data = response.json()


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
        # Extract relevant information from the result_data
        for result in result_data.get('results', []):
            venue_name = result.get('name')
            address = result['location'].get('formatted_address')
            venue_latitude = result['geocodes']['main']['latitude']  # Latitude of the venue
            venue_longitude = result['geocodes']['main']['longitude']  # Longitude of the venue
            categories = [cat['name'] for cat in result['categories']]
            rating = result.get('rating', None)
            total_ratings = result['stats']['total_ratings'] if 'stats' in result else None
        
            # Append extracted data to data_for_df as a dictionary
            data_for_df.append({
                'Original_Latitude': latitude,  # Original latitude from montreal_bike_data
                'Original_Longitude': longitude,  # Original longitude from montreal_bike_data
                'Venue_Name': venue_name,
                'Address': address,
                'Latitude': venue_latitude,  # Latitude of the venue from Foursquare
                'Longitude': venue_longitude,  # Longitude of the venue from Foursquare
                'Categories': categories,
                'Rating': rating,
                'TotalRatings': total_ratings
            })
    
    except requests.exceptions.RequestException as e:
        print(f"Request failed for latitude {latitude}, longitude {longitude}. Error: {e}")

Put your parsed results into a DataFrame

In [ ]:
# Convert the list of dictionaries into a DataFrame
foursquare_df = pd.DataFrame(data_for_df)

# Display the DataFrame
print(foursquare_df.head())

# Save DataFrame to CSV
foursquare_df.to_csv('foursquare_data.csv', index=False)

In [ ]:
#This is updated code from previous commit to deal with missing original latitude and longitude.

import os
from dotenv import load_dotenv
import requests
import pandas as pd

# Load environment variables
load_dotenv('D:/Documents/GitHub/Statistical-Modelling-Project/notebooks/.env')

# Set the Yelp API key
YELP_API_KEY = os.getenv("YELP_API_KEY")
category = 'restaurants'
yelp_url = "https://api.yelp.com/v3/businesses/search"

# Load dataframe from montreal_df.csv to extract latitude and longitude
montreal_bike_data = pd.read_csv('montreal_df.csv')

# Set parameters for Yelp API request
params_yelp = {
    'term': 'restaurant',
    'latitude': None,
    'longitude': None,
    'radius': 1000,  
    'limit': 50
}

# Headers for Yelp API request
headers_yelp = {
    'Authorization': f'Bearer {YELP_API_KEY}'
}

# List to store extracted businesses
stored_business_list = []

# Initialize request count and set maximum requests per day
requests_made = 0
max_requests_per_day = 300

# Function to make Yelp API request
def make_yelp_api_request(params):
    try:
        response_yelp = requests.get(yelp_url, params=params, headers=headers_yelp)
        response_yelp.raise_for_status()
        return response_yelp.json(), response_yelp.status_code
    except requests.exceptions.RequestException as e:
        print(f"Yelp request failed. Error: {e}")
        return None, 0

try:
    # Loop through montreal_bike_data to get latitude and longitude
    for index, row in montreal_bike_data.iterrows():
        if requests_made >= max_requests_per_day:
            print(f"Reached maximum requests per day ({max_requests_per_day}). Stopping further requests.")
            break

        latitude = row['latitude']
        longitude = row['longitude']

        # Update parameters with latitude and longitude from montreal_bike_data
        params_yelp['latitude'] = latitude
        params_yelp['longitude'] = longitude

        # Make Yelp API request
        result_data_yelp, status_code = make_yelp_api_request(params_yelp)
        requests_made += 1

        if status_code == 200:
            businesses = result_data_yelp.get('businesses', [])

            for business in businesses:
                business_info = {
                    'Name': business['name'],
                    'Rating': business['rating'],
                    'Address': ', '.join(business['location']['display_address']),
                    'OriginalLatitude': latitude,  # Store original latitude from montreal_df
                    'OriginalLongitude': longitude,  # Store original longitude from montreal_df
                    'BusinessLatitude': business['coordinates']['latitude'],  # Latitude of the business from Yelp
                    'BusinessLongitude': business['coordinates']['longitude']  # Longitude of the business from Yelp
                }
                stored_business_list.append(business_info)

        elif status_code == 429:
            print(f"Rate limit exceeded. Stopping further requests for today.")
            break
        else:
            print(f"Yelp request failed for latitude {latitude}, longitude {longitude}. Error: {status_code}")

except Exception as e:
    print(f"Exception occurred: {e}")

# Create DataFrame from stored_business_list of dictionaries
yelp_df = pd.DataFrame(stored_business_list)

# Output the DataFrame
print(yelp_df)
yelp_df.to_csv('yelp_data.csv', index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd

# Load environment variables
load_dotenv('D:/Documents/GitHub/Statistical-Modelling-Project/notebooks/.env')

# Set the Yelp API key
YELP_API_KEY = os.getenv("YELP_API_KEY")
category = 'restaurants'
yelp_url = "https://api.yelp.com/v3/businesses/search"

# Load dataframe from montreal_df.csv to extract latitude and longitude
montreal_bike_data = pd.read_csv('montreal_df.csv')

# Set parameters for Yelp API request
params_yelp = {
    'term': 'restaurant',
    'latitude': None,
    'longitude': None,
    'radius': 1000,  
    'limit': 50
}

# Headers for Yelp API request
headers_yelp = {
    'Authorization': f'Bearer {YELP_API_KEY}'
}

# List to store extracted businesses
stored_business_list = []

# Initialize request count and set maximum requests per day
requests_made = 0
max_requests_per_day = 300

# Function to make Yelp API request
def make_yelp_api_request(params):
    try:
        response_yelp = requests.get(yelp_url, params=params, headers=headers_yelp)
        response_yelp.raise_for_status()
        return response_yelp.json(), response_yelp.status_code
    except requests.exceptions.RequestException as e:
        print(f"Yelp request failed. Error: {e}")
        return None, 0

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
try:
    # Loop through montreal_bike_data to get latitude and longitude
    for index, row in montreal_bike_data.iterrows():
        if requests_made >= max_requests_per_day:
            print(f"Reached maximum requests per day ({max_requests_per_day}). Stopping further requests.")
            break

        latitude = row['latitude']
        longitude = row['longitude']

        # Update parameters with latitude and longitude from montreal_bike_data
        params_yelp['latitude'] = latitude
        params_yelp['longitude'] = longitude

        # Make Yelp API request
        result_data_yelp, status_code = make_yelp_api_request(params_yelp)
        requests_made += 1

        if status_code == 200:
            businesses = result_data_yelp.get('businesses', [])

            for business in businesses:
                business_info = {
                    'Name': business['name'],
                    'Rating': business['rating'],
                    'Address': ', '.join(business['location']['display_address']),
                    'OriginalLatitude': latitude,  # Store original latitude from montreal_df
                    'OriginalLongitude': longitude,  # Store original longitude from montreal_df
                    'BusinessLatitude': business['coordinates']['latitude'],  # Latitude of the business from Yelp
                    'BusinessLongitude': business['coordinates']['longitude']  # Longitude of the business from Yelp
                }
                stored_business_list.append(business_info)

        elif status_code == 429:
            print(f"Rate limit exceeded. Stopping further requests for today.")
            break
        else:
            print(f"Yelp request failed for latitude {latitude}, longitude {longitude}. Error: {status_code}")

except Exception as e:
    print(f"Exception occurred: {e}")

Put your parsed results into a DataFrame

In [1]:
# Create DataFrame from stored_business_list of dictionaries
yelp_df = pd.DataFrame(stored_business_list)

# Output the DataFrame
print(yelp_df)
yelp_df.to_csv('yelp_data.csv', index=False)

NameError: name 'pd' is not defined

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

I was able to only get the rating from Yelp Data, even though there was documentation in the Foursquare Places API. There had been a recent change from v2 which used Venues instead of V3 which integrated Venues into Places API

Get the top 10 restaurants according to their rating

In [5]:
top_10_restaurants = yelp_df.sort_values(by='Rating', ascending=False).head(10)

top_10_restaurants

,Name,Rating,Address,Latitude,Longitude
8395,Planète-Pain,5.0,"2020 Rue Gauthier, Montreal, QC H2K 1A7, Canada",45.536505,-73.568039
9774,Kim Quy,5.0,"2098 Rue Jean-Talon E, Montreal, QC H2E 1V3, C...",45.547582,-73.612209
9786,Cambo Thai,5.0,"1660 Rue Jarry E, Montreal, QC H2E 1B3, Canada",45.547582,-73.612209
6347,Sofra Tapas,5.0,"151 Rue Saint Viateur O, Montreal, QC H2T 2L4,...",45.529668,-73.601669
6342,T'Ami Restaurant,5.0,"6542 Boulevard Saint-Laurent, Montreal, QC H2S...",45.529668,-73.601669
9808,Restaurant Poulet Rouge Masson,5.0,"2917 rue Masson, Montreal, QC H1Y 1X5, Canada",45.550114,-73.577222
6326,Restaurant Poulet Rouge St-Denis,5.0,"1750 rue Saint-Denis, Montreal, QC H2X 3K6, Ca...",45.514012,-73.565987
9825,Café Les 3 Jerks,5.0,"2636 Rue de Bellechasse, Montreal, QC H1Y 1J3,...",45.550114,-73.577222
9842,Restaurant Adega,5.0,"3124 Rue Beaubien E, Montreal, QC H1Y 1H3, Canada",45.550114,-73.577222
9850,Forketta Épicerie Gourmet,5.0,"3589 Rue Beaubien E, Montreal, QC H1X 1H1, Canada",45.550114,-73.577222
